In [ ]:
import batoid
import yaml
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
filename = os.path.join(batoid.datadir, 'HSC', 'HSC3.yaml')
config = yaml.load(open(filename))
telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
telescope.draw(ax)
plt.show()

In [ ]:
print([item.name for item in telescope.items])

In [ ]:
# Skip to camera
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
telescope.items[4].draw(ax)
plt.show()

In [ ]:
# Trace some rays and add them to the plots
zdist = 16  # starting height above coordinate origin (which is primary mirror vertex in this case)
outer = 4.1 # radius of PM
inner = 0.5 # camera shadow
xcos = 0.005 # x direction cosine
ycos = 0.003 # y direction cosine
nradii = 10 # number of radii between outer and inner to generate
naz = 10 # number rays around outermost radius
wavelength = 500e-9 # meters
nInit = 1.0 # initial refractive index
rays = batoid.circularGrid(zdist, outer, inner, xcos, ycos, nradii, naz, wavelength, nInit)
tf = telescope.traceFull(rays)

In [ ]:
# tf contains the results from the full raytrace.  The following extracts just the xyz coords of each ray.
# Note that these are initially returned in the coordSys of each optical element, so we have to transform
# them into the global coordinate system to make plots
def paths(tf):
    xs = []
    ys = []
    zs = []
    for item in tf:
        transform = batoid.CoordTransform(item['inCoordSys'], batoid.CoordSys())
        globalRay = transform.applyForward(item['in'])
        xs.append(globalRay.x)
        ys.append(globalRay.y)
        zs.append(globalRay.z)
    return np.array(xs), np.array(ys), np.array(zs)
xs, ys, zs = paths(tf)
print(xs.shape)  # (nsurface, nray)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
telescope.draw(ax)
for x, y, z in zip(np.array(xs).T, np.array(ys).T, np.array(zs).T):
    ax.plot(x, y, z, c='r', alpha=0.3)
ax.scatter(xs, ys, zs, c='r', s=3)    
plt.show()

In [ ]:
# Skip to camera
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
telescope.items[4].draw(ax)
for x, y, z in zip(np.array(xs).T, np.array(ys).T, np.array(zs).T):
    ax.plot(x[5:], y[5:], z[5:], c='r', alpha=0.3)
ax.scatter(xs[5:], ys[5:], zs[5:], c='r', s=3)
plt.show()

In [ ]:
# Interactive
from ipywidgets import interact
import ipywidgets as widgets

@interact(xcos=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          ycos=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0))
def trace(xcos, ycos):
    rays = batoid.circularGrid(
        zdist, outer, inner, 
        1e-2*xcos, 1e-2*ycos, 
        nradii, naz, wavelength, nInit
    )
    tf = telescope.traceFull(rays)
    xs, ys, zs = paths(tf)

    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111, projection='3d')
    telescope.items[4].draw(ax)
    for x, y, z in zip(np.array(xs).T, np.array(ys).T, np.array(zs).T):
        ax.plot(x[5:], y[5:], z[5:], c='r', alpha=0.3)
    ax.scatter(xs[5:], ys[5:], zs[5:], c='r', s=3)
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-0.5, 0.5)
    ax.set_zlim(13.5, 15.25)
    ax.set_xlabel("x")
    ax.set_ylabel("y")